# Understanding primitive inputs and outputs

This page gives an overview of the inputs and outputs of the V2 primitives implemented by Qiskit Runtime which will execute workloads on an IBM Quantum backend. These interfaces provide you with the ability to efficiently define vectorized workloads by using a data structure known as a **Primitive Unified Bloc (PUB)**. These PUBs are used as the fundamental unit of work which a backend needs in order to execute these workloads.

After being submitted as jobs to the IBM Quantum platform, they are executed and the data returned is dependent on the format PUBs which were executed, as well as any error mitigation, suppression, or other runtime options that were defined when the primitive was invoked.

## Overview of PUBS

The PUB data structure defines individual workloads to be submitted to a backend for execution on a QPU. It is a tuple which contains four values:
- A single `QuantumCircuit` which possibly containing one or more `Parameters`
- A list of one or more observables, which specify the expectation values to estimate. The data can be in any one of the `ObservablesArrayLike` format such as `Pauli`, `SparsePauliOp`, or `str`. (Not needed for the `Sampler` primitive)
- A collection of parameter values to bind the circuit against $\theta_k$
- (Optionally) a target precision for expectation values to estimate (Not needed for the `Sampler` primitive)

When invoking a primitive's [`run()`](api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2#qiskit_ibm_runtime.EstimatorV2.run) method, the argument expected is a `list` of one or PUBs, the data of which will depend on the computation being submitted to the backend. 

The below code demonstrates an example set of vectorized inputs to the `Estimator` primitive and executes them on an IBM Quantum backend as a single `RuntimeJobV2 ` object.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    QiskitRuntimeService,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np

# Instantiate runtime service and get
# the least busy backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Define a circuit with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.rz(Parameter("b"), 0)
circuit.cx(0, 1)
circuit.h(0)

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
layout = transpiled_circuit.layout


# Now define a sweep over parameter values, where the two axes are over the
# two parameters in the circuit.
params = np.vstack(
    [
        np.linspace(-np.pi, np.pi, 100),
        np.linspace(-4 * np.pi, 4 * np.pi, 100),
    ]
).T

# Define three observables. The inner length-1 lists cause this array of
# observables to have shape (3, 1), rather than shape (3,) if they were
# omitted.
observables = [
    [SparsePauliOp(["XX", "IY"], [0.5, 0.5])],
    [SparsePauliOp("XX")],
    [SparsePauliOp("IY")],
]
# Apply the same layout as the transpiled circuit.
observables = [
    [observable.apply_layout(layout) for observable in observable_set]
    for observable_set in observables
]

# Estimate the expectation value for all 300 combinations of observables
# and parameter values, where the pub result will have shape (3, 100).
#
# This shape is due to our array of parameter bindings having shape
# (100, 2), combined with our array of observables having shape (3, 1).
estimator_pub = (transpiled_circuit, observables, params)

# Instantiate the new estimator object, then run the transpiled circuit
# using the set of parameters and observables.
estimator = Estimator(mode=backend, options={'resilience_level':0})
job = estimator.run([estimator_pub])
print(job.job_id())
#result = job.result()


In [ ]:
result = service.job('ct63ws37rgf000858ybg').result()


for key, val in result.metadata.items():
    print(f'{key}: {val}')
print(f'PubResult keys: {result[0].data.keys()}')
for key, val in result[0].data.items():
    print(f'{key} : {val}')
#print(f'EVs: {result[0].data.evs}')
#print(f'Standard Deviations: {result[0].data.stds}')
#print(f'Ensemble standard error: {result[0].ensemble_standard_error}') 

## Overview of primitive results

Once one or more PUBs are sent to a QPU for execution and a job is successfully completed, the data is returned as a `PrimitiveResult` container object accessed by calling the `RuntimeJobV2.result()` method. This data structure contains an iterable list of `PubResult` objects, one for every PUB. Each of these `PubResult` objects then contain the execution results of the PUB that was submitted. The data for each `PubResult` is contained within a `DataBin` container object whose attributes are dependent on upon the structure of the associated PUB as well as the error mitigation options specified by the primitive that the job was submitted with (e.g. [ZNE](./run/error-mitigation-explanation#zero-noise-extrapolation-zne) or [PEC](./run/error-mitigation-explanation#probabilistic-error-cancellation-pec)). 

However, no matter what runtime options are specified, each `PubResult` for the **Estimator** primitive will contain a list of expectation values (`PubResult.data.evs`) and associated standard deviations (either `PubResult.data.stds` or `PubResult.data.ensemble_standard_error` depending on the `resilience_level` used).

In [23]:
result = service.job('ct1hh2c1k1wg008rnsr0').result()

print(f'The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n')
print(f'The associated PubResult of this job has the following data:\n {result[0].data}\n')
print(f'And this DataBin has attributes: {result[0].data.keys()}')

The result of the submitted job had 1 PUB and has a value:
 PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(3, 100), dtype=float64>), stds=np.ndarray(<shape=(3, 100), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(3, 100), dtype=float64>), shape=(3, 100)), metadata={'shots': 8192, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 64})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})

The associated PubResult of this job has the following data:
 DataBin(evs=np.ndarray(<shape=(3, 100), dtype=float64>), stds=np.ndarray(<shape=(3, 1

<Admonition type="note" title="Sampler Primitive Output">

The Sampler primitive will output job results in a similar format, with the exception that each `DataBin` will contain one or more `BitArray` objects which store the measured probability distribution(s). The number and attribute label for each bit array object depends on how the `ClassicalRegisters` were defined for the circuit being executed. The measurement data from these `BitArrays` can then be processed into a dictionary with key values pairs corresponding to each bitstring measured (e.g. '1011001') and the number of times (or counts) it was measured.

For example, if a circuit has measurement instructions added by the [`QuantumCircuit.measure_all()`](../api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) function possesses a classical register with the label *'meas'*. After execution, a count data dictionary can be created by executing:

``` python
count_dict = result.data.meas.get_counts()
```

</Admonition>


### Metadata

Once one or more PUBs are sent to a QPU for execution and have successfully completed their associated jobs, the data is returned within a `PrimitiveResult` container object. This data structure contains an iterable list of `PubResult` objects, one for every PUB, as well as a `metadata` attribute. Each of these `PubResult` objects contain the execution results of the PUB that was submitted as well as a `metadata` attribute in the form of a dictionary. The metadata containing information which applies to all the PUBs submitted (such as the various [runtime options](../api/qiskit-ibm-runtime/options) available) can be found in the `PrimitiveResult.metatada` while the metadata which is specific to each PUB is found in `PubResult.metadata`.

In [ ]:
# Print out the results metadata
print(f'The metadata of the result is:')
for key, val in result.metadata.items():
    print(f"'{key}' : {val},")

The metadata of the result is:
'dynamical_decoupling' : {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'},
'twirling' : {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'},
'resilience' : {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False},
'version' : 2,


## Sampler primitive output
For the Sampler primitive, the data returned is in the form of:

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    QiskitRuntimeService,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np

# Instantiate runtime service and get
# the least busy backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(backend)

# Define a circuit with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.rz(Parameter("b"), 0)
circuit.cx(0, 1)
circuit.h(0)


# Now define a sweep over parameter values, where the two axes are over the
# two parameters in the circuit.
params = np.vstack(
    [
        np.linspace(-np.pi, np.pi, 100),
        np.linspace(-4 * np.pi, 4 * np.pi, 100),
    ]
).T

<IBMBackend('ibm_torino')>


In [2]:
# Add measurements to the circuit
circuit.measure_all()

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
layout = transpiled_circuit.layout


# Define the pub for the sampler primitive
sampler_pub = (transpiled_circuit, params)

# Instantiate the new sampler object, then run the transpiled circuit
# using the set of parameters and observables.
sampler = Sampler(mode=backend)
job = sampler.run([sampler_pub])
print(job.job_id())

ct25v5wwtztg008aszng


In [3]:
job = service.job('ct25v5wwtztg008aszng')
result = job.result()

In [15]:
print(f'Sampler Data is: {result[0].data}')
print(f'It has the key-value pair: {result[0].data.items()}')

result[0].data.meas.get_counts()

Sampler Data is: DataBin(meas=BitArray(<shape=(100,), num_shots=4096, num_bits=2>), shape=(100,))
Its keys are: dict_items([('meas', BitArray(<shape=(100,), num_shots=4096, num_bits=2>))])


{'11': 115808, '10': 104963, '01': 89334, '00': 99495}